# Synchronous Interlock CCW/Rotator Integration Test

This notebook performs a synchronous motion interlock scenario integration test between the Camera Cable Wrap (CCW) and the Rotator with the Camera Cable Wrap tracking the Rotator. It includes enough boilerplate to allow the test to run at any time by getting current time information from the pointing and computing appropriate coordinates to slew.

In [ ]:
import logging
import yaml

import numpy as np
from matplotlib import pyplot as plt
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import asyncio
from lsst.ts import salobj
from lsst.ts.idl.enums import ATPtg

from astropy.utils import iers
iers.conf.auto_download = False

In [ ]:
test_message = "Interlock Rotator_CCW Integration Test"

In [ ]:
d = salobj.Domain()

In [ ]:
script = salobj.Controller("Script", index=1)
rot = salobj.Remote(d, "Rotator")
mtptg = salobj.Remote(d, "MTPtg")

In [ ]:
await asyncio.gather(rot.start_task,
                     mtptg.start_task,
                     script.start_task)

In [ ]:
await asyncio.sleep(1.)
await salobj.set_summary_state(mtptg, salobj.State.ENABLED)
await salobj.set_summary_state(rot, salobj.State.ENABLED)

# Move to 0 deg Starting Position

In [ ]:
await asyncio.sleep(0.1)

print("Move to 0.0 deg for starting position")

await rot.cmd_positionSet.set_start(angle=0.0,timeout=10.)

await rot.cmd_move.start(timeout=30.)

tel = rot.tel_Application.get()
cpos = tel.Position
print(cpos)

while cpos > 0.01:
    tel = rot.tel_Application.get()
    cpos = tel.Position

print("Rotator is at 0 deg starting position")

# Set Low Velocity and Move Through Positive Interlock

In [ ]:
await asyncio.sleep(1.)

print("Move to 2.3 deg position")

rot.evt_controllerState.flush()

controllerState = await rot.evt_controllerState.aget()
enabledSubstate = controllerState.enabledSubstate
print("enabledSubstate = " + str(enabledSubstate))

while not enabledSubstate == 0:
    #rot.evt_controllerState.flush()
    controllerState = rot.evt_controllerState.get()
    enabledSubstate = controllerState.enabledSubstate
    #print(enabledSubstate)

await rot.cmd_positionSet.set_start(angle=2.3,timeout=10.)

await rot.cmd_move.start(timeout=30.)

tel = rot.tel_Application.get()
cpos = tel.Position
print(cpos)

while cpos != 2.3:
    tel = rot.tel_Application.get()
    cpos = tel.Position

print("Rotator is at 2.3 deg position")

# Move to Zero

In [ ]:
await asyncio.sleep(1.)

print("Move to 0.0 deg")

rot.evt_controllerState.flush()

controllerState = await rot.evt_controllerState.aget()
enabledSubstate = controllerState.enabledSubstate
print("enabledSubstate = " + str(enabledSubstate))

while enabledSubstate != 0:
    #rot.evt_controllerState.flush()
    controllerState = rot.evt_controllerState.get()
    enabledSubstate = controllerState.enabledSubstate
    #print(enabledSubstate)

await rot.cmd_positionSet.set_start(angle=0.0)

await rot.cmd_move.start(timeout=30.)

tel = rot.tel_Application.get()
cpos = tel.Position

while cpos > 0.01:
    tel = rot.tel_Application.get()
    cpos = tel.Position

print("Rotator is at 0 deg starting position")

# Set Low Velocity and Move Through Negative Interlock

In [ ]:
await asyncio.sleep(1.)

print("Move to -2.3 deg position")

rot.evt_controllerState.flush()

controllerState = await rot.evt_controllerState.aget()
enabledSubstate = controllerState.enabledSubstate
print("enabledSubstate = " + str(enabledSubstate))

while not enabledSubstate == 0:
    #rot.evt_controllerState.flush()
    controllerState = rot.evt_controllerState.get()
    enabledSubstate = controllerState.enabledSubstate
    #print(enabledSubstate)

await rot.cmd_positionSet.set_start(angle=-2.3,timeout=10.)

await rot.cmd_move.start(timeout=30.)

tel = rot.tel_Application.get()
cpos = tel.Position

while cpos != -2.3:
    tel = rot.tel_Application.get()
    cpos = tel.Position

print("Rotator is at -2.3 deg position")

# Move to Zero

In [ ]:
await asyncio.sleep(1.)

print("Move to 0.0 deg")

rot.evt_controllerState.flush()

controllerState = await rot.evt_controllerState.aget()
enabledSubstate = controllerState.enabledSubstate
print("enabledSubstate = " + str(enabledSubstate))

while not enabledSubstate == 0:
    #rot.evt_controllerState.flush()
    controllerState = rot.evt_controllerState.get()
    enabledSubstate = controllerState.enabledSubstate
    #print(enabledSubstate)

await rot.cmd_positionSet.set_start(angle=0.0)

await rot.cmd_move.start(timeout=30.)

tel = rot.tel_Application.get()
cpos = tel.Position

while cpos > 0.01:
    tel = rot.tel_Application.get()
    cpos = tel.Position

print("Test complete")

script.evt_logMessage.set_put(level=logging.INFO+1,
                              message=f"END - {test_message}")